In [1]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
import requests
import warnings

# Suppress the specific warning
warnings.filterwarnings("ignore", category=Warning)

# Number of Detection data

In [2]:
df_carp = pd.read_csv('dc_dets_Carp_summary.csv')
df_central = pd.read_csv('dc_dets_Central_summary.csv')
df_foliage = pd.read_csv('dc_dets_Foliage_summary.csv')
df_telephone = pd.read_csv('dc_dets_Telephone_summary.csv')

df_carp['Location'] = 'Carp Pond'
df_central['Location'] = 'Central Pond'
df_foliage['Location'] = 'Foliage'
df_telephone['Location'] = 'Telephone Field'

column_rename_dict = {
    'num_dets (30of30)': 'num_dets (1800of1800)',
    'num_dets (5of30)': 'num_dets (300of1800)',
    'num_dets (1of6)': 'num_dets (60of360)'
}

df_telephone.rename(columns=column_rename_dict, inplace=True)

df = pd.concat([df_carp, df_central, df_foliage, df_telephone], ignore_index=True)
df.rename(columns={'num_dets (1800of1800)': 'number_of_detection'}, inplace=True)
df = df[['datetime_UTC', 'number_of_detection', 'Location']]
df['datetime_UTC'] = pd.to_datetime(df['datetime_UTC'])
df.rename(columns={'datetime_UTC': 'Datetime UTC'}, inplace=True)
display(df)

,Datetime UTC,number_of_detection,Location
0,2022-07-13 00:00:00,0.0,Carp Pond
1,2022-07-13 00:30:00,0.0,Carp Pond
2,2022-07-13 01:00:00,0.0,Carp Pond
3,2022-07-13 01:30:00,0.0,Carp Pond
4,2022-07-13 02:00:00,0.0,Carp Pond
...,...,...,...
12569,2022-10-17 13:30:00,638.0,Telephone Field
12570,2022-10-17 14:00:00,0.0,Telephone Field
12571,2022-10-17 14:30:00,0.0,Telephone Field
12572,2022-10-17 15:00:00,0.0,Telephone Field


# Weather data

In [3]:
dfw = pd.read_csv('2022-2023-uwa.csv', header=[1])

# Convert 'Date' column to datetime format
dfw['Datetime UTC'] = pd.to_datetime(dfw['Date'] + ' ' + dfw['Time'])

# rename the column "Speed" to "Wind Speed"
dfw = dfw.rename(columns={'Speed': 'Wind Speed'})

dfw2022 = dfw.loc[dfw['Datetime UTC'].dt.year == 2022]

dfw2022['Rain'] = pd.to_numeric(dfw2022['Rain'], errors='coerce')
dfw2022['Wind Speed'] = pd.to_numeric(dfw2022['Wind Speed'], errors='coerce')

display(dfw2022)

,Pseudo-Julian-Date,Date,Time,Temperature,Dewpoint,Relhum,Wind Speed,Gust,Direction,Pressure,Solar,SumRain,Rain,Datetime UTC
0,2.459732e+06,2022-06-01,00:00,66.5,46.6,48.8,8.9,11.0,311,1015.1,517.1,0.00,0.0,2022-06-01 00:00:00
1,2.459732e+06,2022-06-01,00:01,66.6,46.5,48.4,7.1,9.6,292,1015.1,514.9,0.00,0.0,2022-06-01 00:01:00
2,2.459732e+06,2022-06-01,00:02,66.6,46.8,48.8,4.5,5.5,300,1015.1,512.7,0.00,0.0,2022-06-01 00:02:00
3,2.459732e+06,2022-06-01,00:03,66.9,47.1,48.8,4.6,5.3,307,1015.1,510.1,0.00,0.0,2022-06-01 00:03:00
4,2.459732e+06,2022-06-01,00:04,67.4,46.7,47.4,7.9,11.3,296,1015.0,507.4,0.00,0.0,2022-06-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305816,2.459946e+06,2022-12-31,23:55,48.2,42.7,81.0,4.5,6.100000,229,1008.5,27.3,22.67,0.0,2022-12-31 23:55:00
305817,2.459946e+06,2022-12-31,23:56,48.2,42.7,81.1,4.4,5.000000,203,1008.3,27.5,22.67,0.0,2022-12-31 23:56:00
305818,2.459946e+06,2022-12-31,23:57,48.2,42.6,80.8,5.2,6.000000,188,1008.3,27.5,22.67,0.0,2022-12-31 23:57:00
305819,2.459946e+06,2022-12-31,23:58,48.2,42.6,80.8,5.0,7.800000,209,1008.4,27.1,22.67,0.0,2022-12-31 23:58:00


In [4]:
unique_recording_date = df['Datetime UTC'].unique()

save_mean = pd.DataFrame()

for i in range(len(unique_recording_date) - 1):
    start_date = unique_recording_date[i]
    end_date = unique_recording_date[i+1]
    sum_rain = dfw2022.loc[(dfw2022['Datetime UTC'] >= start_date) & (dfw2022['Datetime UTC'] <= end_date), 'Rain'].sum()
    mean_speed = dfw2022.loc[(dfw2022['Datetime UTC'] >= start_date) & (dfw2022['Datetime UTC'] <= end_date), 'Wind Speed'].mean()
    result = {'Datetime UTC': end_date, 'Sum Rain': sum_rain, 'Mean Wind Speed': mean_speed}
    save_mean = save_mean.append(result, ignore_index=True)

display(save_mean)

,Datetime UTC,Sum Rain,Mean Wind Speed
0,2022-07-13 00:30:00,0.0,8.545161
1,2022-07-13 01:00:00,0.0,10.258065
2,2022-07-13 01:30:00,0.0,8.706452
3,2022-07-13 02:00:00,0.0,9.964516
4,2022-07-13 02:30:00,0.0,9.090323
...,...,...,...
3993,2022-07-12 13:30:00,0.0,6.051613
3994,2022-07-12 14:00:00,0.0,6.087097
3995,2022-07-12 14:30:00,0.0,6.451613
3996,2022-07-12 15:00:00,0.0,5.900000


In [5]:
print(save_mean['Sum Rain'].min())
print(save_mean['Sum Rain'].max())
print(save_mean['Mean Wind Speed'].min())
print(save_mean['Mean Wind Speed'].max())

0.0
0.30000000000000004
3.3129032258064512
18.987096774193553


In [6]:
dfw_temp = dfw[['Datetime UTC', 'Temperature']]
df = pd.merge(df, dfw_temp, on='Datetime UTC')

In [7]:
df = pd.merge(df, save_mean, on='Datetime UTC')
display(df)

,Datetime UTC,number_of_detection,Location,Temperature,Sum Rain,Mean Wind Speed
0,2022-07-13 00:30:00,0.0,Carp Pond,77.8,0.0,8.545161
1,2022-07-13 00:30:00,1.0,Foliage,77.8,0.0,8.545161
2,2022-07-13 01:00:00,0.0,Carp Pond,75.4,0.0,10.258065
3,2022-07-13 01:00:00,0.0,Foliage,75.4,0.0,10.258065
4,2022-07-13 01:30:00,0.0,Carp Pond,74.1,0.0,8.706452
...,...,...,...,...,...,...
12557,2022-07-12 13:30:00,1.0,Foliage,62.9,0.0,6.051613
12558,2022-07-12 14:00:00,8.0,Foliage,63.9,0.0,6.087097
12559,2022-07-12 14:30:00,0.0,Foliage,64.1,0.0,6.451613
12560,2022-07-12 15:00:00,4.0,Foliage,66.3,0.0,5.900000


# Lunar Phase data

In [8]:
dflunar = pd.read_csv('NASA_mooninfo_2022.csv', header=[0])

In [9]:
dflunar['Datetime UTC'] = pd.to_datetime(dflunar['Date'] + ' ' + dflunar['Time'].str.strip(' UT'))
dflunar.set_index('Datetime UTC', inplace=True)
dflunar.drop(columns=['Date', 'Time'], inplace=True)

# Perform the resampling. We interpolate with the 'time' method which considers the distance of each time point
# from the surrounding points for a more accurate interpolation.
dflunar_resampled = dflunar.resample('30T').interpolate(method='time')
dflunar_resampled.reset_index(inplace=True)
display(dflunar_resampled)

,Datetime UTC,Phase,Age,Diam,Dist,RA,Dec,Slon,Slat,Elon,Elat,AxisA
0,2022-01-01 00:00:00,4.740,27.6780,1997.10,358876.0,16.91700,-23.88530,-156.6840,-0.9940,-1.8200,1.6960,4.9270
1,2022-01-01 00:30:00,4.630,27.6990,1997.30,358840.0,16.93955,-23.94690,-156.9375,-0.9945,-1.7815,1.7315,4.7990
2,2022-01-01 01:00:00,4.520,27.7200,1997.50,358804.0,16.96210,-24.00850,-157.1910,-0.9950,-1.7430,1.7670,4.6710
3,2022-01-01 01:30:00,4.415,27.7410,1997.70,358769.5,16.98470,-24.06860,-157.4450,-0.9955,-1.7040,1.8025,4.5430
4,2022-01-01 02:00:00,4.310,27.7620,1997.90,358735.0,17.00730,-24.12870,-157.6990,-0.9960,-1.6650,1.8380,4.4150
...,...,...,...,...,...,...,...,...,...,...,...,...
17514,2022-12-31 21:00:00,68.760,8.4470,1842.40,389009.0,1.99190,11.26760,75.0190,-1.3450,7.0930,1.0450,341.3520
17515,2022-12-31 21:30:00,68.955,8.4675,1841.95,389105.0,2.00830,11.38095,74.7660,-1.3455,7.0835,1.0145,341.4055
17516,2022-12-31 22:00:00,69.150,8.4880,1841.50,389201.0,2.02470,11.49430,74.5130,-1.3460,7.0740,0.9840,341.4590
17517,2022-12-31 22:30:00,69.350,8.5090,1841.05,389297.0,2.04115,11.60710,74.2600,-1.3465,7.0640,0.9535,341.5130


In [10]:
df = pd.merge(df, dflunar_resampled, on='Datetime UTC')
display(df)

,Datetime UTC,number_of_detection,Location,Temperature,Sum Rain,Mean Wind Speed,Phase,Age,Diam,Dist,RA,Dec,Slon,Slat,Elon,Elat,AxisA
0,2022-07-13 00:30:00,0.0,Carp Pond,77.8,0.0,8.545161,99.005,13.9010,2005.40,357395.5,18.70975,-26.92245,10.0255,1.3380,-0.8365,5.0050,354.8065
1,2022-07-13 00:30:00,1.0,Foliage,77.8,0.0,8.545161,99.005,13.9010,2005.40,357395.5,18.70975,-26.92245,10.0255,1.3380,-0.8365,5.0050,354.8065
2,2022-07-13 01:00:00,0.0,Carp Pond,75.4,0.0,10.258065,99.050,13.9220,2005.50,357381.0,18.73350,-26.91750,9.7720,1.3380,-0.7930,5.0290,354.6730
3,2022-07-13 01:00:00,0.0,Foliage,75.4,0.0,10.258065,99.050,13.9220,2005.50,357381.0,18.73350,-26.91750,9.7720,1.3380,-0.7930,5.0290,354.6730
4,2022-07-13 01:30:00,0.0,Carp Pond,74.1,0.0,8.706452,99.095,13.9430,2005.55,357368.0,18.75725,-26.91070,9.5180,1.3385,-0.7490,5.0525,354.5400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12557,2022-07-12 13:30:00,1.0,Foliage,62.9,0.0,6.051613,97.645,13.4430,2002.50,357918.0,18.18810,-26.81555,15.6120,1.3295,-1.7790,4.4430,357.7685
12558,2022-07-12 14:00:00,8.0,Foliage,63.9,0.0,6.087097,97.720,13.4640,2002.70,357885.0,18.21170,-26.83020,15.3580,1.3300,-1.7370,4.4700,357.6330
12559,2022-07-12 14:30:00,0.0,Foliage,64.1,0.0,6.451613,97.795,13.4845,2002.85,357854.0,18.23535,-26.84310,15.1040,1.3305,-1.6950,4.4970,357.4980
12560,2022-07-12 15:00:00,4.0,Foliage,66.3,0.0,5.900000,97.870,13.5050,2003.00,357823.0,18.25900,-26.85600,14.8500,1.3310,-1.6530,4.5240,357.3630


In [11]:
summary_stats = df.describe(include='all')
summary_stats

,Datetime UTC,number_of_detection,Location,Temperature,Sum Rain,Mean Wind Speed,Phase,Age,Diam,Dist,RA,Dec,Slon,Slat,Elon,Elat,AxisA
count,12562,10925.000000,12562,12562,12562.00000,12561.000000,12562.000000,12562.000000,12562.000000,12562.000000,12562.000000,12562.000000,12562.000000,12562.000000,12562.000000,12562.000000,12562.000000
unique,3993,NaN,4,635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2022-09-13 09:00:00,NaN,Foliage,59.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,4,NaN,3993,196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,2022-06-15 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2022-10-17 15:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,280.840549,NaN,NaN,0.00014,6.263618,49.691706,14.764405,1863.022966,385288.547484,11.383304,1.984719,-0.112872,1.254938,0.384338,-0.277411,184.205640
std,NaN,979.816138,NaN,NaN,0.00394,1.737405,35.316037,8.405269,72.432875,14879.313586,7.011283,19.222560,101.968063,0.299134,4.388943,4.679940,165.518911
min,NaN,0.000000,NaN,NaN,0.00000,3.312903,0.090000,0.005000,1762.800000,357267.000000,0.003600,-27.402300,-179.992000,0.517500,-7.423000,-6.711000,0.005000
25%,NaN,0.000000,NaN,NaN,0.00000,4.954839,13.921250,7.545000,1790.600000,371526.000000,5.301400,-17.116850,-83.222000,1.076500,-3.830000,-4.895000,16.485000


In [13]:
df['Month'] = df['Datetime UTC'].dt.month
df['Day'] = df['Datetime UTC'].dt.day
df['Hour'] = df['Datetime UTC'].dt.hour

In [ ]:
df.to_csv('weather_clean_df.csv', index=False)